# 1) Comparing Police Districts:
* Which district has the most crimes? Which has the least?

# 2) Crimes Across the Years:
* Is the total number of crimes increasing or decreasing across the years?
* Are there any individual crimes that are doing the opposite (e.g decreasing when overall crime is increasing or vice-versa)?

# 4) Comparing Months:
* What months have the most crime? What months have the least?
* Are there any individual crimes that do not follow this pattern? If so, which crimes?